# **Vall-e.** 


In [ ]:
#@title Install Vall-e dependencies
# PyTorch
!pip install torch==1.13.1 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install torchmetrics==0.11.1
# fbank
!pip install librosa==0.8.1

# phonemizer
!apt-get install espeak-ng
## OSX: brew install espeak
!pip install phonemizer

# lhotse
# https://github.com/lhotse-speech/lhotse/pull/956
# https://github.com/lhotse-speech/lhotse/pull/960
!pip uninstall lhotse
!pip uninstall lhotse
!pip install git+https://github.com/lhotse-speech/lhotse

# k2 icefall
# pip install k2
!git clone https://github.com/k2-fsa/k2.git
!cd k2
!export K2_MAKE_ARGS="-j12"
!export K2_CMAKE_ARGS="-DK2_WITH_CUDA=OFF"
!python setup.py install
!cd -

!git clone https://github.com/k2-fsa/icefall
!cd icefall
!pip install -r requirements.txt
!export PYTHONPATH=`pwd`/../icefall:$PYTHONPATH
!echo "export PYTHONPATH=`pwd`/../icefall:\$PYTHONPATH" >> ~/.zshrc
!echo "export PYTHONPATH=`pwd`/../icefall:\$PYTHONPATH" >> ~/.bashrc
!cd -

# valle
!git clone https://github.com/lifeiteng/valle.git
!cd valle
!pip install -e .

In [ ]:
#@title Prepare Dataset
!pip install librosa==0.8.1
!cd egs/libritts

# Those stages are very time-consuming
!bash run.sh --stage -1 --stop-stage 3

In [ ]:
#@title Train
# 12G GPU --max-duration 24 --num-decoder-layers 6
bash run.sh --stage 4 --stop-stage 4 \
    --num-decoder-layers 12 \
    --max-duration 40 --use-fp16 true

In [ ]:
#@title Inference
!python3 bin/infer.py \
    --decoder-dim 1024 --nhead 16 --num-decoder-layers 12 --model-name valle \
    --text-prompts "Go to her." \
    --audio-prompts ./prompts/61_70970_000007_000001.wav \
    --text "To get up and running quickly just follow the steps below." \
    --output-dir infer/demos \
    --checkpoint exp/valle/epoch-10.pt